In [1]:
from Carmudi import Carmudi #class news Scrapper

In [2]:
obj = Carmudi()

In [3]:
obj.getAllBerita([], 1)

page  1
https://www.carmudi.co.id/journal/page/1
memasukkan berita id  38254
Insert berita  Harga Lexus ES Terbaru di Indonesia Lebih Murah Dibandingkan di Thailand
masuk
memasukkan berita id  38252
Insert berita  Tips Merawat Mesin Turbo : Yang Penting Rajin Ganti Oli dan Service Berkala
masuk
memasukkan berita id  38227
Insert berita  Stres Saat Mengemudi di Jalan, Ini Cara Mudah Mengatasinya
masuk
memasukkan berita id  38238
Insert berita  Aston Martin DB5 James Bond Kembali Diproduksi dengan ‘Gadget’ Khas 007
masuk


'berhasil ambil semua berita'